# Testing

In [1]:
!pip install polars

     |████████████████████████████████| 15.1 MB 1.5 MB/s eta 0:00:01


In [67]:
import pandas as pd
import polars as pl

In [68]:
%%timeit
df = pd.read_csv("/home/wmnlab/D/database/2022-09-29/_Bandlock_Udp/sm05/#01/analysis/clt_dwnlnk_udp_packet_info.csv",
                 dtype={'repackage.num':'int8'})
# display(df.head())
df = df.dropna(subset=['repackage.num'])
# print((df['repackage.num'] > 1).sum())
df.index[df['repackage.num'] > 1]

162 ms ± 850 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [102]:
%%timeit
df = pl.read_csv("/home/wmnlab/D/database/2022-09-29/_Bandlock_Udp/sm05/#01/analysis/clt_dwnlnk_udp_packet_info.csv") \
    .with_columns(pl.col('repackage.num').cast(pl.Int8, strict=False))
# display(df.head())
df = df.filter(~pl.col('repackage.num').is_null())
# print((df['repackage.num'] > 1).sum())
df.to_pandas().index[(df['repackage.num'] > 1).to_pandas()]

183
183
183
94.2 ms ± 21.8 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


# Detect

In [1]:
import os
from os.path import isdir, isfile
import polars as pl
import pandas as pd
from tqdm.notebook import tqdm

def detect_repackage(filepath, tags):
    ### polars
    # # print('hello')
    # df = pl.read_csv(filepath) \
    #     .with_columns(pl.col('repackage.num').cast(pl.Int8, strict=False))
    # df = df.select('repackage.num')
    # # print('hello1')
    # df = df.filter(~pl.col('repackage.num').is_null())
    # N = len(df)
    # if N == 0:
    #     return
    # # print('hello2')
    # df_rpk1 = df['repackage.num'] > 1
    # # print('hello3')
    # if df_rpk1.sum() > 0:
    #     # print(filepath)
    #     print(f'{filepath}: row {df.to_pandas().index[df_rpk1.to_pandas()][0]}, total {df_rpk1.sum()}/{N}, {round(df_rpk1.sum()/N*100, 2)}%')
    # # print('hello4')
    ### pandas
    # print('hello')
    df = pd.read_csv(filepath,
        dtype={'repackage.num':'int8'})
    df = df['repackage.num']
    # print('hello1')
    df = df.dropna()
    N = len(df)
    if N == 0:
        return
    # print('hello2')
    df_rpk1 = df > 1
    # print('hello3')
    if df_rpk1.sum() > 0:
        # print(filepath)
        print(f'{filepath}: row {df.index[df_rpk1][0]}, total {df_rpk1.sum()}/{N}, {round(df_rpk1.sum()/N*100, 2)}%')
    # print('hello4')
    return

def dfs(visited, node, filetags, tags, level=0):  # function for dfs
    global pbar
    if node in visited:
        return
    visited.add(node)
    ### TODO: 'node' is an absolute pathname (string) for a file or a directory.
    if os.path.basename(node).startswith(filetags):
        try:
            pbar.update(1)
            if pbar.n % 20 == 0:
                print("process is still alive!")
            detect_repackage(node, tags)
        except:
            print(f'{node}: processing error!')
    ### TODO END
    if isfile(node):
        return
    children = [os.path.join(node, s) for s in sorted(os.listdir(node))]
    for child in children:
        dfs(visited, child, filetags, tags, level+1)
    ### print message in the end
    if level == 0:
        print("**********************************************************************")

In [2]:
# Client Downlink
root = "/home/wmnlab/D/database"
visited = set()  # Set to keep track of visited nodes of graph (tree).
pbar = tqdm(total=48_763)
dfs(visited, root, filetags=('clt_dwnlnk_udp_packet_info.csv','udp_dnlk_client_pkt_info.csv'), tags=[])

# /tmp/ipykernel_74252/819301199.py:27: DtypeWarning: Columns (9,11) have mixed types. Specify dtype option on import or set low_memory=False.
# df = pd.read_csv(filepath,
# 應該是在講 sequence.number 和 payload.time_epoch
# sequence.number: 8557(int)  8558@8559(str)
# payload.time_epoch: 1672045053.819261(float)  1672045053.821261@1672045053.823261(str)
# 但無傷大雅，這份 code 用不到它們兩位

  0%|          | 0/48763 [00:00<?, ?it/s]

/home/wmnlab/D/database/2022-09-29/_Bandlock_Udp/sm05/#01/analysis/clt_dwnlnk_udp_packet_info.csv: row 0, total 183/27860, 0.66%
/home/wmnlab/D/database/2022-09-29/_Bandlock_Udp/sm05/#02/analysis/clt_dwnlnk_udp_packet_info.csv: row 1, total 42033/43853, 95.85%
/home/wmnlab/D/database/2022-09-29/_Bandlock_Udp/sm05/#03/analysis/clt_dwnlnk_udp_packet_info.csv: row 1, total 44649/46324, 96.38%
/home/wmnlab/D/database/2022-09-29/_Bandlock_Udp/sm05/#04/analysis/clt_dwnlnk_udp_packet_info.csv: row 1, total 42459/44127, 96.22%
/home/wmnlab/D/database/2022-09-29/_Bandlock_Udp/sm05/#05/analysis/clt_dwnlnk_udp_packet_info.csv: row 0, total 44763/47105, 95.03%
/home/wmnlab/D/database/2022-09-29/_Bandlock_Udp/sm05/#06/analysis/clt_dwnlnk_udp_packet_info.csv: row 0, total 41621/43886, 94.84%
/home/wmnlab/D/database/2022-09-29/_Bandlock_Udp/sm06/#01/analysis/clt_dwnlnk_udp_packet_info.csv: row 19, total 163/27053, 0.6%
/home/wmnlab/D/database/2022-09-29/_Bandlock_Udp/sm06/#02/analysis/clt_dwnlnk_udp_

/tmp/ipykernel_74793/2735628477.py:27: DtypeWarning: Columns (9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath,


/home/wmnlab/D/database/2022-12-26/_Modem_Phone_Comparative_Exeriments/sm07/#06/middle/udp_dnlk_client_pkt_info.csv: row 2, total 42202/227222, 18.57%
process is still alive!
/home/wmnlab/D/database/2022-12-26/_Modem_Phone_Comparative_Exeriments/sm08/#01/middle/udp_dnlk_client_pkt_info.csv: row 1, total 64840/73388, 88.35%
/home/wmnlab/D/database/2022-12-26/_Modem_Phone_Comparative_Exeriments/sm08/#02/middle/udp_dnlk_client_pkt_info.csv: row 1, total 51231/58023, 88.29%
/home/wmnlab/D/database/2022-12-26/_Modem_Phone_Comparative_Exeriments/sm08/#03/middle/udp_dnlk_client_pkt_info.csv: row 1, total 46201/52521, 87.97%
/home/wmnlab/D/database/2022-12-26/_Modem_Phone_Comparative_Exeriments/sm08/#04/middle/udp_dnlk_client_pkt_info.csv: row 1, total 45475/51676, 88.0%
/home/wmnlab/D/database/2022-12-26/_Modem_Phone_Comparative_Exeriments/sm08/#05/middle/udp_dnlk_client_pkt_info.csv: row 1, total 52730/59924, 87.99%


/tmp/ipykernel_74793/2735628477.py:27: DtypeWarning: Columns (9,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath,


/home/wmnlab/D/database/2022-12-26/_Modem_Phone_Comparative_Exeriments/sm08/#06/middle/udp_dnlk_client_pkt_info.csv: row 1, total 46184/232117, 19.9%
/home/wmnlab/D/database/2023-01-01/_MRT_Greenline/sm00/#01/middle/udp_dnlk_client_pkt_info.csv: row 0, total 82055/95478, 85.94%
/home/wmnlab/D/database/2023-01-01/_MRT_Greenline/sm00/#02/middle/udp_dnlk_client_pkt_info.csv: row 1, total 78384/93079, 84.21%
process is still alive!
/home/wmnlab/D/database/2023-01-01/_MRT_Greenline/sm00/#03/middle/udp_dnlk_client_pkt_info.csv: row 0, total 80394/93403, 86.07%
/home/wmnlab/D/database/2023-01-01/_MRT_Greenline/sm00/#04/middle/udp_dnlk_client_pkt_info.csv: row 0, total 75052/88471, 84.83%
/home/wmnlab/D/database/2023-01-01/_MRT_Greenline/sm01/#01/middle/udp_dnlk_client_pkt_info.csv: row 0, total 81027/95488, 84.86%
/home/wmnlab/D/database/2023-01-01/_MRT_Greenline/sm01/#02/middle/udp_dnlk_client_pkt_info.csv: row 0, total 77902/92284, 84.42%
/home/wmnlab/D/database/2023-01-01/_MRT_Greenline/sm

In [3]:
# Client Uplink
root = "/home/wmnlab/D/database"
visited = set()  # Set to keep track of visited nodes of graph (tree).
pbar = tqdm(total=48_763)
dfs(visited, root, filetags=('clt_uplnk_udp_packet_info.csv','udp_uplk_client_pkt_info.csv'), tags=[])

  0%|          | 0/48763 [00:00<?, ?it/s]

process is still alive!
process is still alive!
process is still alive!
process is still alive!
process is still alive!
process is still alive!
process is still alive!
process is still alive!
process is still alive!
process is still alive!
process is still alive!
process is still alive!
process is still alive!
process is still alive!
**********************************************************************


In [4]:
# Server Downlink
root = "/home/wmnlab/D/database"
visited = set()  # Set to keep track of visited nodes of graph (tree).
pbar = tqdm(total=48_763)
dfs(visited, root, filetags=('srv_dwnlnk_udp_packet_info.csv','udp_dnlk_server_pkt_info.csv'), tags=[])

  0%|          | 0/48763 [00:00<?, ?it/s]

process is still alive!
process is still alive!
process is still alive!
process is still alive!
process is still alive!
process is still alive!
process is still alive!
process is still alive!
process is still alive!
process is still alive!
process is still alive!
process is still alive!
process is still alive!
process is still alive!
**********************************************************************


In [5]:
# Server Uplink
root = "/home/wmnlab/D/database"
visited = set()  # Set to keep track of visited nodes of graph (tree).
pbar = tqdm(total=48_763)
dfs(visited, root, filetags=('srv_uplnk_udp_packet_info.csv','udp_uplk_server_pkt_info.csv'), tags=[])

  0%|          | 0/48763 [00:00<?, ?it/s]

process is still alive!
process is still alive!
process is still alive!
process is still alive!
process is still alive!
process is still alive!
process is still alive!
process is still alive!
process is still alive!
process is still alive!
process is still alive!
process is still alive!
process is still alive!
process is still alive!
**********************************************************************


: 